### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
%load_ext lab_black

In [2]:
from functools import reduce

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv(
    "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat",
    delimiter="::",
    header=None,
    names=["movie_id", "movie", "genre"],
    dtype={"movie_id": object},
    engine="python",
)
reviews = pd.read_csv(
    "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat",
    delimiter="::",
    header=None,
    names=["user_id", "movie_id", "rating", "timestamp"],
    dtype={"movie_id": object, "user_id": object, "timestamp": object},
    engine="python",
)

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [3]:
# cell for work
print(len(movies))
print(len(reviews))

37163
902147


In [4]:
l = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
from functools import reduce

k = reduce(lambda l1, l2: l1 + l2, l)
print(k)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [5]:
# cell for work
genres = list(movies.query("genre.notna()").genre.str.split("|"))
k = reduce(lambda l1, l2: l1 + l2, genres)

In [6]:
len(set(k))

28

In [7]:
# more work
reviews.user_id.nunique()

70431

In [8]:
# and still more
reviews.isna().sum()

user_id      0
movie_id     0
rating       0
timestamp    0
dtype: int64

In [9]:
# and even more - open additional cells as necessary
reviews.rating.describe()

count    902147.000000
mean          7.316031
std           1.852799
min           0.000000
25%           6.000000
50%           8.000000
75%           9.000000
max          10.000000
Name: rating, dtype: float64

In [10]:
# Use your findings to match each variable to the correct statement in the dictionary
dict_sol1 = {
    "The number of movies in the dataset": 37163,
    "The number of ratings in the dataset": 902147,
    "The number of different genres": 28,
    "The number of unique users in the dataset": 70431,
    "The number missing ratings in the reviews dataset": 0,
    "The average rating given across all ratings": 7.316031,
    "The minimum rating given across all ratings": 0,
    "The maximum rating given across all ratings": 10,
}

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp
* Create month/day/year columns from timestamp as dummies

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [11]:
# cell for work
movies = movies.assign(
    year=lambda x: x.movie.str.slice(-5, -1),
    movie=lambda x: x.movie.str.slice(0, -6),
)
movies["1800's"] = movies.year.str.startswith("18")
movies["1900's"] = movies.year.str.startswith("19")
movies["2000's"] = movies.year.str.startswith("20")
movies = pd.concat([movies, pd.get_dummies(movies.genre)], axis=1)

In [12]:
# and another
reviews = reviews.assign(date=lambda x: pd.to_datetime(x.timestamp, unit="s"))

In [13]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date
0           0        1    114508       8  1381006850  2013-10-05 23:00:50
1           1        2    499549       9  1376753198  2013-08-17 17:26:38
2           2        2   1305591       8  1376742507  2013-08-17 14:28:27
3           3        2   1428538       1  1371307089  2013-06-15 16:38:09
4           4        3     75314       1  1595468524  2020-07-23 03:42:04
   Unnamed: 0  movie_id                                              movie  \
0           0         8      Edison Kinetoscopic Record of a Sneeze (1894)   
1           1        10                La sortie des usines Lumière (1895)   
2           2        12                      The Arrival of a Train (1896)   
3           3        25  The Oxford and Cambridge University Boat Race ...   
4           4        91                         Le manoir du diable (1896)   

               genre  date  1800's  1900's  2000's  Thriller  Family  ...  \
0  Documen